In [1]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem

plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["figure.dpi"] = 300


sys.path.append("scripts")
sys.path.append("../../scripts")

In [2]:
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from dim_reduction import SpectralEmbedding, scBiMapEmbedding,GaussianRandomProjection,SparseRandomProjection,SimHash_Dimredu
from nearest_neighbors import (
    ExactNearestNeighbors,
    NNDescent,
    WeightedLowHash,
    PAFNearestNeighbors,
    LowHash,
    HNSW,
    ProductQuantization,
    _NearestNeighbors,
    IVFProductQuantization,
    SimHash,
    BlockSimHash,
    NewSimHash,
    NewSimHash2,
    NewSimHash3
)
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors
from plots import plot_read_graph, mp_plot_read_graphs, get_graphviz_layout, get_umap_layout

/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [7]:
find_neighbor_method = []
methods = ['Exact_Euclidean','Exact_Cosine',
          'NNDescent_Euclidean','PQ_Euclidean','PQ_Cosine','HNSW_Euclidean','HNSW_Cosine',
          'LowHash_Jaccard','WeightedLowHash_Jaccard','MinHash_Jaccard','WeightedMinHash_Jaccard','NNDescent_Euclidean']
dim_redu_method = ['Spectural_100d','Spectural_500d','scBiMap_100d','scBiMap_500d','None']
pre_process = ['TF','IDF','TF-IDF','None']
for a in methods:
    for b in dim_redu_method:
        for c in pre_process:
            simple_name = '_'.join([a,b,c])
            find_neighbor_method.append(simple_name)
            

simple_to_complete = {'Exact':'ExactNearestNeighbors',
      'PQ':'ProductQuantization',
      'scBiMap':'scBiMapEmbedding',
      'Spectural':'SpectralEmbedding',      
      '100d':'100',
      '500d':'500'}

simple_names = []
configs_str = []
for method_index in range(len(find_neighbor_method)):
    simple_name = find_neighbor_method[method_index]
    simple_name_list = simple_name.split('_')
    for i in range(len(simple_name_list)):
        if simple_name_list[i] in simple_to_complete:
            simple_name_list[i] = simple_to_complete[simple_name_list[i]]
    complete_name = simple_name_list
    des = ','.join(complete_name)
    if 'LowHash' in complete_name[0] and complete_name[2] == 'None':
        simple_names.append(simple_name)
        config = f'''NearestNeighborsConfig(
        nearest_neighbors_method={complete_name[0]},
        description='{des}',
        tfidf='{complete_name[3]}',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))'''
        configs_str.append(config)
        
    elif 'MinHash' in complete_name[0] and complete_name[2] == 'None':
        if 'Weight' in complete_name[0]:
            simple_names.append(simple_name)
            config = f'''NearestNeighborsConfig(
            nearest_neighbors_method=WeightedLowHash,
            description='{des}',
            tfidf='{complete_name[3]}',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))'''
            configs_str.append(config)
        else:
            simple_names.append(simple_name)
            config = f'''NearestNeighborsConfig(
            nearest_neighbors_method=LowHash,
            description='{des}',
            tfidf='{complete_name[3]}',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))'''
            configs_str.append(config)
    elif complete_name[0] in ['HNSW','ProductQuantization','NNDescent','ExactNearestNeighbors']:
        if complete_name[2] == 'None':
            config = f'''NearestNeighborsConfig(
            nearest_neighbors_method={complete_name[0]},
            description='{des}',
            tfidf='{complete_name[3]}',
            nearest_neighbors_kw=dict(metric='{complete_name[1].lower()}')) '''
            simple_names.append(simple_name)
            configs_str.append(config)
        else:
            config = f'''NearestNeighborsConfig(
            nearest_neighbors_method={complete_name[0]},
            description='{des}',
            tfidf='{complete_name[4]}',
            dimension_reduction_method={complete_name[2]},
            dimension_reduction_kw=dict(n_dimensions={complete_name[3]}),
            nearest_neighbors_kw=dict(metric='{complete_name[1].lower()}')) '''
            simple_names.append(simple_name)
            configs_str.append(config)

In [8]:
for i in range(len(simple_names)):
    add_to_dict = 'config_dict["'+simple_names[i]+'"]='+configs_str[i]
    print(add_to_dict)

config_dict["Exact_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description=

In [3]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_bucket_size = COVERAGE_DEPTH * 1.5
read_features = []
config_dict = {}
config_dict["Exact_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Cosine_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["NNDescent_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["PQ_Cosine_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["PQ_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ProductQuantization,
            description='ProductQuantization,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Cosine_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["LowHash_Jaccard_None_TF"]=NearestNeighborsConfig(
        nearest_neighbors_method=LowHash,
        description='LowHash,Jaccard,None,TF',
        tfidf='TF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["LowHash_Jaccard_None_IDF"]=NearestNeighborsConfig(
        nearest_neighbors_method=LowHash,
        description='LowHash,Jaccard,None,IDF',
        tfidf='IDF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["LowHash_Jaccard_None_TF-IDF"]=NearestNeighborsConfig(
        nearest_neighbors_method=LowHash,
        description='LowHash,Jaccard,None,TF-IDF',
        tfidf='TF-IDF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["LowHash_Jaccard_None_None"]=NearestNeighborsConfig(
        nearest_neighbors_method=LowHash,
        description='LowHash,Jaccard,None,None',
        tfidf='None',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["WeightedLowHash_Jaccard_None_TF"]=NearestNeighborsConfig(
        nearest_neighbors_method=WeightedLowHash,
        description='WeightedLowHash,Jaccard,None,TF',
        tfidf='TF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["WeightedLowHash_Jaccard_None_IDF"]=NearestNeighborsConfig(
        nearest_neighbors_method=WeightedLowHash,
        description='WeightedLowHash,Jaccard,None,IDF',
        tfidf='IDF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["WeightedLowHash_Jaccard_None_TF-IDF"]=NearestNeighborsConfig(
        nearest_neighbors_method=WeightedLowHash,
        description='WeightedLowHash,Jaccard,None,TF-IDF',
        tfidf='TF-IDF',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["WeightedLowHash_Jaccard_None_None"]=NearestNeighborsConfig(
        nearest_neighbors_method=WeightedLowHash,
        description='WeightedLowHash,Jaccard,None,None',
        tfidf='None',
        nearest_neighbors_kw=dict(
        lowhash_fraction=0.01,
        max_bucket_size=max_bucket_size,
        repeats=100,
        seed=458,
        ))
config_dict["MinHash_Jaccard_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=LowHash,
            description='MinHash,Jaccard,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["MinHash_Jaccard_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=LowHash,
            description='MinHash,Jaccard,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["MinHash_Jaccard_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=LowHash,
            description='MinHash,Jaccard,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["MinHash_Jaccard_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=LowHash,
            description='MinHash,Jaccard,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["WeightedMinHash_Jaccard_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=WeightedLowHash,
            description='WeightedMinHash,Jaccard,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["WeightedMinHash_Jaccard_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=WeightedLowHash,
            description='WeightedMinHash,Jaccard,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["WeightedMinHash_Jaccard_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=WeightedLowHash,
            description='WeightedMinHash,Jaccard,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["WeightedMinHash_Jaccard_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=WeightedLowHash,
            description='WeightedMinHash,Jaccard,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(
            lowhash_count=20,
            max_bucket_size=max_bucket_size,
            repeats=100,
            seed=458,
            ))
config_dict["NNDescent_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["NNDescent_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NNDescent,
            description='NNDescent,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["SimHash_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=SimHash,
            description='SimHash,Cosine,None,None',
            tfidf='None')
config_dict["SimHash_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=SimHash,
            description='SimHash,Cosine,None,TF',
            tfidf='TF') 
config_dict["SimHash_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=SimHash,
            description='SimHash,Cosine,None,IDF',
            tfidf='IDF') 
config_dict["SimHash_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=SimHash,
            description='SimHash,Cosine,None,TF-IDF',
            tfidf='TF-IDF') 
config_dict["NewSimHash_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=NewSimHash,
            description='NewSimHash,Cosine,None,None',
            tfidf='None')
config_dict["NewSimHash_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NewSimHash,
            description='NewSimHash,Cosine,None,TF',
            tfidf='TF') 
config_dict["NewSimHash_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NewSimHash,
            description='NewSimHash,Cosine,None,IDF',
            tfidf='IDF') 
config_dict["NewSimHash_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=NewSimHash,
            description='NewSimHash,Cosine,None,TF-IDF',
            tfidf='TF-IDF') 

config_dict["BlockSimHash_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=BlockSimHash,
            description='BlockSimHash,Cosine,None,None',
            tfidf='None')
config_dict["BlockSimHash_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=BlockSimHash,
            description='BlockSimHash,Cosine,None,TF',
            tfidf='TF') 
config_dict["BlockSimHash_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=BlockSimHash,
            description='BlockSimHash,Cosine,None,IDF',
            tfidf='IDF') 
config_dict["BlockSimHash_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=BlockSimHash,
            description='BlockSimHash,Cosine,None,TF-IDF',
            tfidf='TF-IDF') 

#-----HNSW and random projection
config_dict["HNSW_Euclidean_GaussianRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,None',
            tfidf='None',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 

config_dict["HNSW_Euclidean_SparseRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,None',
            tfidf='None',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 

#-----exact and random projection

config_dict["Exact_Euclidean_GaussianRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,None',
            tfidf='None',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 

config_dict["Exact_Euclidean_SparseRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,None',
            tfidf='None',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 


config_dict["Exact_Euclidean_SimHash_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SimHash_Dimredu,3200,None',
            tfidf='None',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SimHash_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SimHash_Dimredu,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SimHash_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SimHash_Dimredu,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_SimHash_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SimHash_Dimredu,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 

config_dict["HNSW_Euclidean_SimHash_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SimHash_Dimredu,3200,None',
            tfidf='None',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SimHash_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SimHash_Dimredu,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SimHash_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SimHash_Dimredu,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_SimHash_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SimHash_Dimredu,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='euclidean')) 

config_dict["Exact_Cosine_SimHash_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Cosine,SimHash_Dimredu,3200,None',
            tfidf='None',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_SimHash_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Cosine,SimHash_Dimredu,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_SimHash_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Cosine,SimHash_Dimredu,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_SimHash_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Cosine,SimHash_Dimredu,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 

config_dict["HNSW_Cosine_SimHash_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SimHash_Dimredu,3200,None',
            tfidf='None',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_SimHash_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SimHash_Dimredu,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_SimHash_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SimHash_Dimredu,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_SimHash_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SimHash_Dimredu,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SimHash_Dimredu,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='cosine')) 

In [4]:
import pickle
with open('/home/miaocj/docker_dir/kNN-overlap-finder/workflow/notebooks/config_dict.pkl', 'wb') as file:  
    pickle.dump(config_dict, file) 

In [8]:
config_dict["BlockSimHash_Cosine_None_TF-IDF"]

NearestNeighborsConfig(description='BlockSimHash,Cosine,None,TF-IDF', tfidf='TF-IDF', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.BlockSimHash'>)

In [36]:
with open('/home/miaocj/docker_dir/kNN-overlap-finder/workflow/notebooks/config_dict.pkl', 'rb') as file:  
    config_dict1 = pickle.load(file) 
config_dict1["NNDescent_Euclidean_Spectural_500d_TF-IDF"]

NearestNeighborsConfig(description='NNDescent,Euclidean,SpectralEmbedding,500,TF-IDF', tfidf='TF-IDF', dimension_reduction_method=<class 'dim_reduction.SpectralEmbedding'>, nearest_neighbors_method=<class 'nearest_neighbors.NNDescent'>)

In [42]:
mytest1 = []
for method in simple_names:
    if '100' not in method and 'scBiMap' in method:
        if 'HNSW' in method:
            mytest1.append(method)
        elif 'PQ' in method:
            mytest1.append(method)
        elif 'Exact' in method:
            mytest1.append(method)
print(mytest1)
len(mytest1)

['Exact_Euclidean_scBiMap_500d_TF', 'Exact_Euclidean_scBiMap_500d_IDF', 'Exact_Euclidean_scBiMap_500d_TF-IDF', 'Exact_Euclidean_scBiMap_500d_None', 'Exact_Cosine_scBiMap_500d_TF', 'Exact_Cosine_scBiMap_500d_IDF', 'Exact_Cosine_scBiMap_500d_TF-IDF', 'Exact_Cosine_scBiMap_500d_None', 'PQ_Euclidean_scBiMap_500d_TF', 'PQ_Euclidean_scBiMap_500d_IDF', 'PQ_Euclidean_scBiMap_500d_TF-IDF', 'PQ_Euclidean_scBiMap_500d_None', 'PQ_Cosine_scBiMap_500d_TF', 'PQ_Cosine_scBiMap_500d_IDF', 'PQ_Cosine_scBiMap_500d_TF-IDF', 'PQ_Cosine_scBiMap_500d_None', 'HNSW_Euclidean_scBiMap_500d_TF', 'HNSW_Euclidean_scBiMap_500d_IDF', 'HNSW_Euclidean_scBiMap_500d_TF-IDF', 'HNSW_Euclidean_scBiMap_500d_None', 'HNSW_Cosine_scBiMap_500d_TF', 'HNSW_Cosine_scBiMap_500d_IDF', 'HNSW_Cosine_scBiMap_500d_TF-IDF', 'HNSW_Cosine_scBiMap_500d_None']


24

In [16]:
mytest2 = []
for method in simple_names:
    if 'TF-IDF' in method:
        if '100' not in method and 'scBiMap' in method:
            if 'Exact' in method:
                mytest2.append(method)
        elif 'Min' in method or 'Low' in method:
            mytest2.append(method)
        elif 'Exact' in method and 'None_' in method:
            mytest2.append(method)
mytest2

['Exact_Euclidean_scBiMap_500d_TF-IDF',
 'Exact_Euclidean_None_TF-IDF',
 'Exact_Cosine_scBiMap_500d_TF-IDF',
 'Exact_Cosine_None_TF-IDF',
 'LowHash_Jaccard_None_TF-IDF',
 'WeightedLowHash_Jaccard_None_TF-IDF',
 'MinHash_Jaccard_None_TF-IDF',
 'WeightedMinHash_Jaccard_None_TF-IDF']

In [20]:
mytest2 = []
for method in simple_names:
    if 'TF-IDF' in method and ('500' in method or 'Hash' in method):
        mytest2.append(method)
mytest2

['Exact_Euclidean_Spectural_500d_TF-IDF',
 'Exact_Euclidean_scBiMap_500d_TF-IDF',
 'Exact_Cosine_Spectural_500d_TF-IDF',
 'Exact_Cosine_scBiMap_500d_TF-IDF',
 'NNDescent_Euclidean_Spectural_500d_TF-IDF',
 'NNDescent_Euclidean_scBiMap_500d_TF-IDF',
 'PQ_Euclidean_Spectural_500d_TF-IDF',
 'PQ_Euclidean_scBiMap_500d_TF-IDF',
 'PQ_Cosine_Spectural_500d_TF-IDF',
 'PQ_Cosine_scBiMap_500d_TF-IDF',
 'HNSW_Euclidean_Spectural_500d_TF-IDF',
 'HNSW_Euclidean_scBiMap_500d_TF-IDF',
 'HNSW_Cosine_Spectural_500d_TF-IDF',
 'HNSW_Cosine_scBiMap_500d_TF-IDF',
 'LowHash_Jaccard_None_TF-IDF',
 'WeightedLowHash_Jaccard_None_TF-IDF',
 'MinHash_Jaccard_None_TF-IDF',
 'WeightedMinHash_Jaccard_None_TF-IDF',
 'NNDescent_Euclidean_Spectural_500d_TF-IDF',
 'NNDescent_Euclidean_scBiMap_500d_TF-IDF']